# Laptop specifications scrapper
Scrap comparez-malin.com for laptop specification

In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import qgrid
import numpy as np
from IPython.core.interactiveshell import InteractiveShell

from laptop_scoring import scrap
from laptop_scoring.utils import scale, ease, display_laptop


# Display all variables alon in a line
InteractiveShell.ast_node_interactivity = "all"

## Get all specs needed

In [ ]:
#Get links to each laptop page in a  dataframe
df_laptops_urls = scrap.get_laptops_urls()

# Get specs for all laptops
#df_laptops_urls = df_laptops_urls.sample(n=100, random_state=0)
df = scrap.get_all_laptops_specs(df_laptops_urls, overwrite=False)
df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace(".", "").str.replace("(", "").str.replace(")", "")

# Get CPU and GPU benchmarks
df_cpu = scrap.get_cpu_dataframe(overwrite=False)
df = df.reset_index().merge(df_cpu, on="processeur", how="left").set_index("index")

df_gpu = scrap.get_gpu_dataframe(overwrite=False)
df = df.reset_index().merge(df_gpu, on="puce_graphique_dédiée", how="left").set_index("index")
df["puce_graphique_dédiée"] = df["puce_graphique_dédiée"].str.replace("Nvidia", "").str.replace("GeForce", "").str.replace("GTX", "")

## Processing & Cleaning

In [ ]:
# One-liners
df["prix_public"] = df["prix_public"].str.strip("€").str.replace(" ", "").str.replace(",", ".").astype(float)
df["prix"] = df["prix"].str.strip("€").str.replace(" ", "").str.replace(",", ".").astype(float)
df[["coeurs", "min_freq", "max_freq"]] = df["fréquence"].str.split(expand=True)[[0,2,4]].astype(float)
df["single_core_benchmark"] = df["cpu_benchmark"] / df["coeurs"]
df["pdt_max"] = df["pdt_max"].str.split(expand=True)[0].astype(int)
df["mémoire_ram"] = df["mémoire_ram"].str.split(expand=True)[0].astype(int)

# Split disque_dur in sshd (bool), hdd_size, hdd_speed, ssd_size
df["sshd"] = df["disque_dur"].apply(lambda x: ("cache SSD" in x))
df["disque_dur"] = df["disque_dur"].str.replace("cache SSD", "").str.replace("(", "").str.replace(")", "")
df["hdd_size"] = df["disque_dur"].apply(lambda x: int(x.split("tr/min")[0].split()[0]) if ("tr/min" in x) else 0)
df["hdd_speed"] = df["disque_dur"].apply(lambda x: int((x.split("tr/min")[0].strip().split()[-1])) if ("tr/min" in x) else 0)
df["ssd_size"] = df["disque_dur"].apply(lambda x: int(x.split("tr/min")[-1].split("Go SSD")[0].split()[-1].replace("SSD", "")) if ("Go SSD" in x) else 0)

df["res_width"] = df["résolution"].str.split(expand=True)[0].astype(int)
df["taille"] = df["taille"].str.split('" ', expand=True)[0].astype(float)
df[["width", "depth", "height"]] = df["dimensions"].str.split(expand=True)[[0,2,4]].astype(float)
df[["width", "depth", "height"]] = df["dimensions"].str.split("x", expand=True)
# Sometimes heights are written "17 - 18" so take the max
df["height"] = df["height"].str.replace(" mm", "").str.split("-", expand=True).fillna(0).astype(float).max(axis=1)
df[["width", "depth"]] = df[["width", "depth"]].astype(float)
df["poids"] = df["poids"].str.replace("kg", "").str.replace("g", "").str.strip().astype(float)
# Convert weights expressed in grams to kilograms
df["poids"] = df["poids"].apply(lambda x: x if x<100 else x/1000)
df = df.fillna(0)

## Scoring

In [ ]:
# Wether to use official price (might not be available) or current price
#prix = "prix_public"
prix = "prix"
if prix == "prix":
    # Remove laptops with no current price (often because unavailable)
    df = df[(df["prix"]!=0)]

In [ ]:
# List of tuples: (col_name, method, do_scale)
scoring_methods = [
    (prix, lambda x: (x/800) ** 2, False),
    ("single_core_benchmark", lambda x: x * 1, True),
    ("coeurs", lambda x: x * 1, True),
    ("mémoire_ram", lambda x: x * 0.5, True),
    ("gpu_benchmark", lambda x: ease(x) * 1, True),
    ("min_freq", lambda x: x * 0.3, True),
    ("max_freq", lambda x: x * 0.5, True),
    ("pdt_max", lambda x: x * -0.5, True),
    ("hdd_speed", lambda x: (x==7200) * 0.5, False),
    ("sshd", lambda x: x * 0.6, False),
    ("ssd_size", lambda x: ((x>0) + ease(x))/2 * 0.6, True),
    ("poids", lambda x: x * -0.5, True),
    ("height", lambda x: -x * 0.8, True),
    ("taille", lambda x: (x>15.6 or x<13.3) * -2 + (x<14) * -1, False),
    ("res_width", lambda x: ease(x) * 1, True),
    ("composition", lambda x: (("Aluminium" in x) or ("Métal" in x)) * 1, False),
    ("marque", lambda x: (x=="Acer") * -0.5, False)
]

df_score = pd.DataFrame(index=df.index, columns=["score"])


# Generic methods that take one column as input
# and output its associated score
for (col_name, method, do_scale) in tqdm(scoring_methods):
    col = df[col_name]
    if do_scale:
        col = scale(col)
    df_score[col_name] = col.apply(method)

# Compute total score
df_score = df_score.fillna(0)
df_score["total"] = df_score.drop(["score", prix], axis=1).sum(axis=1)
df_score["total"] /= df_score[prix]
df_score["score"] = df_score["total"]/df_score["total"].max()
df["score"] = df_score["score"]
df = df.sort_values(by="score", ascending=False)
df_score = df_score.loc[df.index]

## Display results

In [ ]:
#qgrid.set_defaults(grid_options={'forceFitColumns': False})#, 'defaultColumnWidth': 200})
cols_to_show = "url marque référence score taille res_width processeur puce_graphique_dédiée mémoire_ram ssd_size prix prix_public".split()
#height single_core_benchmark coeurs cpu_benchmark gpu_benchmark min_freq max_freq pdt_max hdd_speed sshd 
qgrid.show_grid(df[cols_to_show])

In [ ]:
# Display top ranked computer web pages
iframe = '<iframe width="100%" height="350" src="{}"></iframe>'
for index, row in df.head().iterrows():
    print("-" * 50)
    #print(df_score.loc[index])
    display_laptop(row["url"])
    break